In [18]:
#Importing necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
import torch
from torch import nn
from scipy.stats import pearsonr
from torch.utils.data import DataLoader, TensorDataset

Maybe the 50 dimensions is a little too small and I should start with 128

In [2]:
model = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/embedding_model')

In [3]:
# Load your fine-tuned local model 
model_path = '/home/mendu/Thesis/data/musiccaps/embedding_model'
model = SentenceTransformer(model_path)

In [5]:
# captions = pd.read_csv('/home/mendu/Thesis/data/musiccaps/lp-music-caps_caption_writing.csv', index_col = [0])

In [6]:
msd_dataset = load_dataset('seungheondoh/LP-MusicCaps-MSD')

In [7]:
train = pd.DataFrame(msd_dataset['train'])
# test = pd.DataFrame(msd_dataset['test'])
# valid = pd.DataFrame(msd_dataset['valid'])

In [8]:
sentences = train['caption_writing'].to_list()

In [9]:
# Generate embeddings for your sentences using the fine-tuned model
embedded_sentences = model.encode(sentences, show_progress_bar=True, convert_to_numpy=True)

# Number of PCA components (e.g., reduce to 50 dimensions)
num_components = 50
obj = PCA(n_components=num_components)

# Fit the PCA model to the embedded sentences (this will find the principal components)
pca_embeddings = obj.fit_transform(embedded_sentences)

Batches:   0%|          | 0/13903 [00:00<?, ?it/s]

In [11]:
# Project the PCA embeddings back to the original space
projected_embeddings = obj.inverse_transform(pca_embeddings)

# Initialize an empty list to store the Pearson correlation coefficients
pearsons_correlations = []

# Calculate Pearson's correlation for each pair of original and projected embeddings
for original, projected in zip(embedded_sentences, projected_embeddings):
    # Compute Pearson's r
    corr, _ = pearsonr(original, projected)
    pearsons_correlations.append(corr)

# If you want to compute a single Pearson's correlation coefficient for all data
# Concatenate all embeddings and compute the correlation
flat_original = embedded_sentences.flatten()
flat_projected = projected_embeddings.flatten()
overall_corr, _ = pearsonr(flat_original, flat_projected)

# print("Pearsons correlation for each embedding pair:", pearsons_correlations)
print("Overall Pearson's correlation:", overall_corr)

Overall Pearson's correlation: 0.9783695752137767


In [12]:
print("Original embeddings shape:", embedded_sentences.shape)
print("PCA-reduced embeddings shape:", pca_embeddings.shape)

Original embeddings shape: (444865, 768)
PCA-reduced embeddings shape: (444865, 50)


In [13]:
# import torch
# from torch import nn

# class Autoencoder(nn.Module):
#     def __init__(self, input_size, encoding_size):
#         super(Autoencoder, self).__init__()
#         # Encoder
#         self.encoder = nn.Sequential(
#             nn.Linear(input_size, encoding_size),
#             nn.ReLU(True)
#         )
#         # Decoder
#         self.decoder = nn.Sequential(
#             nn.Linear(encoding_size, input_size),
#             nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
#         )

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded
    

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, encoding_size),
            nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, input_size),
            nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [19]:
# Convert the NumPy array of embeddings to a PyTorch tensor
embedded_sentences_tensor = torch.tensor(embedded_sentences, dtype=torch.float32)

# Create a dataset and a dataloader
dataset = TensorDataset(embedded_sentences_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

In [20]:
# Instantiate the autoencoder
input_size = embedded_sentences.shape[1]
encoding_size = 50  # change this to whatever size you want to encode down to
autoencoder = Autoencoder(input_size=input_size, encoding_size=encoding_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

In [21]:
input_size

768

In [22]:
epochs = 50  # Set this to the number of epochs to train for
for epoch in range(epochs):
    for data in dataloader:
        inputs = data[0]
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 0.3475065529346466
Epoch 2/50, Loss: 0.3487062454223633
Epoch 3/50, Loss: 0.34524354338645935
Epoch 4/50, Loss: 0.3496750295162201
Epoch 5/50, Loss: 0.35347533226013184
Epoch 6/50, Loss: 0.3394722044467926
Epoch 7/50, Loss: 0.33730897307395935
Epoch 8/50, Loss: 0.3398774564266205
Epoch 9/50, Loss: 0.34341609477996826
Epoch 10/50, Loss: 0.33544039726257324
Epoch 11/50, Loss: 0.3458317220211029
Epoch 12/50, Loss: 0.3407270014286041
Epoch 13/50, Loss: 0.3472829759120941
Epoch 14/50, Loss: 0.3407555818557739
Epoch 15/50, Loss: 0.3482276201248169
Epoch 16/50, Loss: 0.3465031385421753
Epoch 17/50, Loss: 0.3476271629333496
Epoch 18/50, Loss: 0.348853200674057
Epoch 19/50, Loss: 0.34133246541023254
Epoch 20/50, Loss: 0.34861934185028076
Epoch 21/50, Loss: 0.3498746454715729
Epoch 22/50, Loss: 0.34424281120300293
Epoch 23/50, Loss: 0.3405202329158783
Epoch 24/50, Loss: 0.34666380286216736
Epoch 25/50, Loss: 0.34207960963249207
Epoch 26/50, Loss: 0.34857282042503357
Epoch 27/50

In [23]:
# Switch the autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the decoded (projected) embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach().numpy()
decoded_embeddings = autoencoder.decoder(torch.from_numpy(encoded_embeddings)).detach().numpy()

# Calculate Pearson's correlation
from scipy.stats import pearsonr
correlations = np.array([pearsonr(original, decoded)[0] for original, decoded in zip(embedded_sentences, decoded_embeddings)])

# print('Pearson correlation for each embedding pair:', correlations)
print('Mean Pearson correlation:', np.mean(correlations))

Mean Pearson correlation: 0.7770308981632065


Change the metric, reconstruction loss

In [24]:
print("Original embeddings shape:", embedded_sentences_tensor.shape)

# Switch autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the encoded embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach()

print("Entire dataset encoded embeddings shape:", encoded_embeddings.shape)

Original embeddings shape: torch.Size([444865, 768])
Entire dataset encoded embeddings shape: torch.Size([444865, 50])
